In [1]:
%run ./robustTestLib.ipynb

2022-10-27 11:56:31.937408: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
en_acc = np.array([0]*10,dtype=float)
e_acc = np.array([0]*10,dtype=float) 
m_acc = np.array([0]*10,dtype=float)
l_acc = np.array([0]*10,dtype=float)
run = 0

In [3]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 0 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmp724c5deq/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmp724c5deq/train/Ae-albopictus/ = 1408.
No. of files in "/tmp/tmp724c5deq/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmp724c5deq/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmp724c5deq/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmp724c5deq/val/Ae-albopictus/ = 132.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmp724c5deq/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmp724c5deq/train/An-tessellatus/ = 1221.
No. of files in "/tmp/tmp724c5deq/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmp724c5deq/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmp724c5deq/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmp724c5deq/val/An-tessellatus/

2022-10-27 11:57:26.987707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 11:57:28.004234: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22280 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Found 5037 files belonging to 7 classes.
./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5 is being loaded.


2022-10-27 11:57:43.922457: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


  5/630 [..............................] - ETA: 25s - loss: 2.2172 - accuracy: 0.5750   

2022-10-27 11:57:47.004024: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


630/630 [==============================] - 22s 26ms/step - loss: 1.6917 - accuracy: 0.7040
A model is deleted.
./models/Model_Middle_Combine_all_Vivo-V21_2022-10-22_07-50.h5 is being loaded.
630/630 [==============================] - 18s 27ms/step - loss: 1.2660 - accuracy: 0.7798
A model is deleted.
./models/Model_Late_Combine_all_Vivo-V21_2022-10-22_09-23.h5 is being loaded.
630/630 [==============================] - 20s 31ms/step - loss: 1.5005 - accuracy: 0.7411
Found 3357 files belonging to 7 classes.
Found 3357 files belonging to 7 classes.
Found 3357 files belonging to 7 classes.
A model is deleted.
./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5 is being loaded.
420/420 [==============================] - 10s 22ms/step
A model is deleted.
./models/Model_Middle_Combine_all_Vivo-V21_2022-10-22_07-50.h5 is being loaded.
420/420 [==============================] - 11s 25ms/step
A model is deleted.
./models/Model_Late_Combine_all_Vivo-V21_2022-10-22_09-23.h5 is being loa

In [5]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 1 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmpkfvb2iol/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmpkfvb2iol/train/Ae-albopictus/ = 1397.
No. of files in "/tmp/tmpkfvb2iol/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmpkfvb2iol/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmpkfvb2iol/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmpkfvb2iol/val/Ae-albopictus/ = 143.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmpkfvb2iol/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmpkfvb2iol/train/An-tessellatus/ = 1210.
No. of files in "/tmp/tmpkfvb2iol/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmpkfvb2iol/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmpkfvb2iol/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmpkfvb2iol/val/An-tessellatus/

In [6]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 2 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmptxdd7cgr/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmptxdd7cgr/train/Ae-albopictus/ = 1408.
No. of files in "/tmp/tmptxdd7cgr/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmptxdd7cgr/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmptxdd7cgr/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmptxdd7cgr/val/Ae-albopictus/ = 132.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmptxdd7cgr/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmptxdd7cgr/train/An-tessellatus/ = 1232.
No. of files in "/tmp/tmptxdd7cgr/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmptxdd7cgr/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmptxdd7cgr/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmptxdd7cgr/val/An-tessellatus/

In [7]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 3 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmp0_b73ov0/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmp0_b73ov0/train/Ae-albopictus/ = 1474.
No. of files in "/tmp/tmp0_b73ov0/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmp0_b73ov0/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmp0_b73ov0/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmp0_b73ov0/val/Ae-albopictus/ = 66.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmp0_b73ov0/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmp0_b73ov0/train/An-tessellatus/ = 1232.
No. of files in "/tmp/tmp0_b73ov0/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmp0_b73ov0/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmp0_b73ov0/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmp0_b73ov0/val/An-tessellatus/ 

In [8]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 4 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmp2n4y65qz/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmp2n4y65qz/train/Ae-albopictus/ = 1408.
No. of files in "/tmp/tmp2n4y65qz/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmp2n4y65qz/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmp2n4y65qz/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmp2n4y65qz/val/Ae-albopictus/ = 132.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmp2n4y65qz/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmp2n4y65qz/train/An-tessellatus/ = 1221.
No. of files in "/tmp/tmp2n4y65qz/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmp2n4y65qz/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmp2n4y65qz/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmp2n4y65qz/val/An-tessellatus/

In [9]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 5 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmpgc7w9ejf/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmpgc7w9ejf/train/Ae-albopictus/ = 1419.
No. of files in "/tmp/tmpgc7w9ejf/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmpgc7w9ejf/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmpgc7w9ejf/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmpgc7w9ejf/val/Ae-albopictus/ = 121.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmpgc7w9ejf/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmpgc7w9ejf/train/An-tessellatus/ = 1254.
No. of files in "/tmp/tmpgc7w9ejf/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmpgc7w9ejf/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmpgc7w9ejf/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmpgc7w9ejf/val/An-tessellatus/

In [10]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 6 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmp0s4imvzk/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmp0s4imvzk/train/Ae-albopictus/ = 1397.
No. of files in "/tmp/tmp0s4imvzk/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmp0s4imvzk/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmp0s4imvzk/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmp0s4imvzk/val/Ae-albopictus/ = 143.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmp0s4imvzk/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmp0s4imvzk/train/An-tessellatus/ = 1210.
No. of files in "/tmp/tmp0s4imvzk/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmp0s4imvzk/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmp0s4imvzk/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmp0s4imvzk/val/An-tessellatus/

In [11]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 7 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmpu9ch0doh/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmpu9ch0doh/train/Ae-albopictus/ = 1397.
No. of files in "/tmp/tmpu9ch0doh/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmpu9ch0doh/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmpu9ch0doh/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmpu9ch0doh/val/Ae-albopictus/ = 143.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmpu9ch0doh/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmpu9ch0doh/train/An-tessellatus/ = 1287.
No. of files in "/tmp/tmpu9ch0doh/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmpu9ch0doh/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmpu9ch0doh/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmpu9ch0doh/val/An-tessellatus/

In [12]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 8 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmpr3y8km13/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmpr3y8km13/train/Ae-albopictus/ = 1419.
No. of files in "/tmp/tmpr3y8km13/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmpr3y8km13/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmpr3y8km13/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmpr3y8km13/val/Ae-albopictus/ = 121.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmpr3y8km13/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmpr3y8km13/train/An-tessellatus/ = 1221.
No. of files in "/tmp/tmpr3y8km13/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmpr3y8km13/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmpr3y8km13/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmpr3y8km13/val/An-tessellatus/

In [13]:
# Sometimes kernel dies, we have to save accuracy on a file every run, load it up to continue next run.
if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print('Next run: ' + str(run))
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))
if run==10:
    exit()
print("------------- Run " + str(run) + " -------------------")
    
# Only 90% will be used, so that, for each run reshuffle data makes different pairs and groups of three mosquito images.
split_LR = tempfile.TemporaryDirectory().name
split_LMR = tempfile.TemporaryDirectory().name
randomSplitMosData(input_dir = mos_img_dir, output_dirLR = split_LR, output_dirLMR = split_LMR)
#------------------------
# Read the dataset, with ratio = (0.90, 0.1, 0.00) most images are in the train dataset, but will be used to test the accuracy.
# All the models were trained by images photographed with proper a camera holder and good lighting, 
# but all images here were taken by someone who hold the camera by hands and no additional photo lighting.
# Moreover, all the images have not been seen by the models.
# To reuse the code with minimum modification, val_ds, and its derivations are still present. 
(train_ds, val_ds, train_ds1, val_ds1) = read_ds(split_LR)
(train_ds_X_X1_Y, val_ds_X_X1_Y) = combineDS(train_ds, val_ds, train_ds1, val_ds1)
#---------------------
# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)
# evaluate with not so good images
loss, e_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)
# evaluate with not so good images
loss, m_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)
# evaluate with not so good images
loss, l_acc[run] = model.evaluate(train_ds_X_X1_Y)
#---------------------
del train_ds, val_ds, train_ds1, val_ds1
del train_ds_X_X1_Y, val_ds_X_X1_Y

# use the same dataset but instead of making pairs of images, make groups of three images (same species.)
(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR) = read_dsLMR(split_LMR)
(train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y,
            ) = combineDS_LMR(train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR)
#---------------------

# recall the early-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print( earlyModelPath + ' is being loaded.')
model = keras.models.load_model(earlyModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainE, Yh_valE) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the middle-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(middleModelPath + ' is being loaded.')
model = keras.models.load_model(middleModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainM, Yh_valM) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)
#---------------------

# recall the late-combined model (A), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(lateModelPath + ' is being loaded.')
model = keras.models.load_model(lateModelPath)

# Don't evaluate but predict so that the model yields confidence, which will be one of the ensemble's inputs
(Yh_trainL, Yh_valL) = predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y)

#---------------------

Yh_train = np.hstack((Yh_trainE, Yh_trainM))
Yh_train = np.hstack((Yh_train, Yh_trainL))
Y_train = np.concatenate([y for (x,y) in train_ds_XL_XM_Y], axis=0)

# recall the ensemble model (D), trained with very good images
if 'model' in globals(): 
    del model
    tf.keras.backend.clear_session()
    print('A model is deleted.')
print(ensemblePath + ' is being loaded.')
model = keras.models.load_model(ensemblePath)
loss, en_acc[run] = model.evaluate(Yh_train,Y_train)

#---------------------
Y_pred = model.predict(Yh_train)
y_pred = np.argmax(Y_pred, axis=1)
Y_true = Y_train
y_true = np.argmax(Y_true, axis=1)

print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('')
print('')
print('Classification Report')

print(classification_report(y_true, y_pred, target_names=class_names))
del model
tf.keras.backend.clear_session()
print('(A) accuracy: ' + str(e_acc[run]))
print('(B) accuracy: ' + str(m_acc[run]))
print('(C) accuracy: ' + str(l_acc[run]))
print('(D) accuracy: ' + str(en_acc[run]))
run += 1
np.savez('./accs',en_acc=en_acc,e_acc=e_acc, m_acc=m_acc, l_acc=l_acc, run=run)
del train_dsL, val_dsL, train_dsM, val_dsM, train_dsR, val_dsR
del train_ds_XL_XM_Y, val_ds_XL_XM_Y, train_ds_XM_XL_Y, val_ds_XM_XL_Y
del train_ds_XM_XR_Y, val_ds_XM_XR_Y, train_ds_XR_XM_Y, val_ds_XR_XM_Y
del train_ds_XL_XR_Y, val_ds_XL_XR_Y, train_ds_XR_XL_Y, val_ds_XR_XL_Y
del Yh_trainE, Yh_valE, Yh_trainM, Yh_valM, Yh_trainL, Yh_valL
del Yh_train, Y_train
gc.collect()

if run == 10:
    exit()

------------- Run 9 -------------------
Ratio = (0.9, 0.1, -0.0).
Found Ae-albopictus
No. of files in Ae-albopictus = 1540.
No. of files in "/tmp/tmphzif0ewu/train/Ae-albopictus/" should be around 1386.
No. of files in /tmp/tmphzif0ewu/train/Ae-albopictus/ = 1408.
No. of files in "/tmp/tmphzif0ewu/test/Ae-albopictus/" should be around 0.
No. of files in /tmp/tmphzif0ewu/test/Ae-albopictus/ = 0.
No. of files in "/tmp/tmphzif0ewu/val/Ae-albopictus/" should be around 154.
No. of files in /tmp/tmphzif0ewu/val/Ae-albopictus/ = 132.
Found An-tessellatus
No. of files in An-tessellatus = 1342.
No. of files in "/tmp/tmphzif0ewu/train/An-tessellatus/" should be around 1208.
No. of files in /tmp/tmphzif0ewu/train/An-tessellatus/ = 1243.
No. of files in "/tmp/tmphzif0ewu/test/An-tessellatus/" should be around 0.
No. of files in /tmp/tmphzif0ewu/test/An-tessellatus/ = 0.
No. of files in "/tmp/tmphzif0ewu/val/An-tessellatus/" should be around 135.
No. of files in /tmp/tmphzif0ewu/val/An-tessellatus/

In [1]:
import numpy as np

if 'run' not in globals():
    %run ./robustTestLib.ipynb
    saved = np.load('./accs.npz')
    en_acc = saved['en_acc']
    e_acc = saved['e_acc']
    m_acc = saved['m_acc']
    l_acc = saved['l_acc']
    run = saved['run']
    print(run)
    print('e_acc = ' + str(e_acc[:run]))
    print('m_acc = ' + str(m_acc[:run]))
    print('l_acc = ' + str(l_acc[:run]))
    print('en_acc = ' + str(en_acc[:run]))


2022-10-27 15:55:02.705839: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


10
e_acc = [0.70399046 0.69777602 0.69833398 0.71970743 0.71368212 0.69320196
 0.70248759 0.71263683 0.7026757  0.70686233]
m_acc = [0.77982926 0.76826847 0.76874256 0.78118205 0.77505028 0.76197046
 0.77393037 0.77233833 0.76457667 0.78778261]
l_acc = [0.74111575 0.73828435 0.74335581 0.74896222 0.74909455 0.73339903
 0.73532337 0.73970151 0.74540734 0.74811584]
en_acc = [0.85939825 0.86352801 0.85892856 0.86884272 0.85929954 0.85249776
 0.85692948 0.86298507 0.86073673 0.87087178]


In [7]:

print('Mean (A): ' + str(round(np.mean(e_acc)*100,2)) + '%')
print('SD (A)  : ' + str(round(np.std(e_acc)*100,2)) + '%')
print('Mean (B): ' + str(round(np.mean(m_acc)*100,2)) + '%')
print('SD (B)  : ' + str(round(np.std(m_acc)*100,2)) + '%')
print('Mean (C): ' + str(round(np.mean(l_acc)*100,2)) + '%')
print('SD (C)  : ' + str(round(np.std(l_acc)*100,2)) + '%')
print('Mean (D): ' + str(round(np.mean(en_acc)*100,2)) + '%')
print('SD (D)  : ' + str(round(np.std(en_acc)*100,2)) + '%')

Mean (A): 70.51%
SD (A)  : 0.78%
Mean (B): 77.34%
SD (B)  : 0.75%
Mean (C): 74.23%
SD (C)  : 0.54%
Mean (D): 86.14%
SD (D)  : 0.52%
